# ESSAI SMAC Session
Welcome to the Euro PhD School “Data Science meets Combinatorial Optimization” session on SMAC! In the following we will guide you throught the SVM configuration example from the [SMAC repository](https://github.com/automl/SMAC3). You can also read up more details in the [SMAC documentation](https://automl.github.io/SMAC3/v2.0.1/index.html).

## SMAC Installation



We need to install conda for colab first and then proceed with the regular smac install as described in the [SMAC](https://github.com/automl/SMAC3/tree/main) repository.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:21
🔁 Restarting kernel...


In [ ]:
!conda create -n SMAC python=3.10
!conda activate SMAC
#!conda install gxx_linux-64 gcc_linux-64 swig
!pip install smac

**Important**: After the installation above finished, you have to restart the runtime!

## A Small Example To Validate the Installation

We now test in the following, if the installation worked by executing a tiny example.

In [ ]:
from ConfigSpace import Configuration, ConfigurationSpace

import numpy as np
from smac import HyperparameterOptimizationFacade, Scenario
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

iris = datasets.load_iris()


def train(config: Configuration, seed: int = 0) -> float:
    classifier = SVC(C=config["C"], random_state=seed)
    scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
    return 1 - np.mean(scores)


configspace = ConfigurationSpace({"C": (0.100, 1000.0)})

# Scenario object specifying the optimization environment
scenario = Scenario(configspace, deterministic=True, n_trials=200)

# Use SMAC to find the best configuration/hyperparameters
smac = HyperparameterOptimizationFacade(scenario, train)
incumbent = smac.optimize()

## A Classical Training Pipeline

Congratulations. Now you have SMAC at your disposal. Let's give it a test ride.
The task is to optimize the hyperparameters of a simple SVM on the Iris dataset. Usually we have some training pipeline with a dataset, a configured model and some validation procedure to check for generalization performance like this:

In [ ]:
from sklearn import datasets, svm

# We load the iris-dataset (a widely used benchmark)
iris = datasets.load_iris()

def train_svm()->float:
  classifier = svm.SVC(degree=3, kernel='poly', C= 1.0, shrinking=True,  gamma='auto')
  scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
  cost = 1-np.mean(scores)

  return cost



In [ ]:
print(train_svm())

## Setting up the Search Space

This training pipeline is a hardcoded version of our training pipeline, where we as experts already have configured a lot manually - and probably spent considerable time and effort in setting these hyperparameters, but have most likely not found a truly good choice of hyperparameters leading to reliable and reproducible results.

To alleviate this situation, we want to use SMAC to automatically configure these hyperparameters for us. For this purpose, we need to define what choices SMAC should optimize over. We can express this using the [ConfigSpace](https://automl.github.io/ConfigSpace/main/) package.

Let us start by selecting both the `kernel` and `C` parameter, just to get a first look into ConfigSpace:

In [ ]:
from ConfigSpace import Categorical, Configuration, ConfigurationSpace, Float, Integer
from ConfigSpace.conditions import InCondition


# define the search space object
cs = ConfigurationSpace(seed=0)  # The seed is mostly important for the sampling which we will see in a second.

# define the hyperparameters with their default values
kernel= Categorical("kernel", ["linear", "poly", "rbf", "sigmoid"], default="poly")
C = Float("C", (0.001, 1000.0), default=1.0, log=True) # Notice, that we define the penalty weight on a log scale!

# register the hyperparameters in the search space
cs.add_hyperparameters([kernel, C])


A nice thing about the `ConfigurationSpace` object is that it already checks the bounds of our parameters and lets us sample the search space. You can execute the following cell several times to see that it truly samples from the space and does not always return the same configuration.


In [ ]:
config = cs.sample_configuration(1)
print(config)

Of course, we will have to adjust our pipeline to accomodate this newly gained flexibility, like we would when we wrote a script with argparse:

In [ ]:
def train_svm(config:Configuration)->float:
  config_dict = dict(config)
  classifier = svm.SVC(degree=3, kernel=config_dict['kernel'], C=config_dict['C'], shrinking=True,  gamma='auto')
  scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
  cost = 1-np.mean(scores)

  return cost

In [ ]:
train_svm(config)

This is a nice place to start. Now we can do a lot more elaborate stuff with the ConfigSpace package such as dealing with different types of hyperparameters (e.g. Categorical, Integer and Float or even setting up conditional hyperparameters; i.e. hyperparameters whose occurance is depending on whether or not another hyperparameter is active.) Let us formulate a rather broad search space with a few of these dependencies:

In [ ]:
cs = ConfigurationSpace(seed=0)

# First we create our hyperparameters
kernel = Categorical("kernel", ["linear", "poly", "rbf", "sigmoid"], default="poly")
C = Float("C", (0.001, 1000.0), default=1.0, log=True)
shrinking = Categorical("shrinking", [True, False], default=True)
degree = Integer("degree", (1, 5), default=3)
coef = Float("coef0", (0.0, 10.0), default=0.0)
gamma = Categorical("gamma", ["auto", "value"], default="auto")
gamma_value = Float("gamma_value", (0.0001, 8.0), default=1.0, log=True)

# Then we create dependencies
use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef = InCondition(child=coef, parent=kernel, values=["poly", "sigmoid"])
use_gamma = InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"])
use_gamma_value = InCondition(child=gamma_value, parent=gamma, values=["value"])

# Add hyperparameters and conditions to our configspace
cs.add_hyperparameters([kernel, C, shrinking, degree, coef, gamma, gamma_value])
cs.add_conditions([use_degree, use_coef, use_gamma, use_gamma_value])



In [ ]:
cs.sample_configuration(5) # and sample n configurations

Of course we  again need to adjust our training pipeline accordingly.
Notice, that the gamma parameter is a bit more tricky; we can either set it to `value` or to `auto` in case of `auto` the SVM model will determine the value of gamma automatically, wheres `value` give us users the opportunity to set it ourselves - which we in turn need to set. This is why we introduced the `gamma_value` variable earlier.

You can see this added complexity in the `svm.SVC` documentation:
> gamma{‘scale’, ‘auto’} or float, default=’scale’
> Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
>
>if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) >as value of gamma,
>
>if ‘auto’, uses 1 / n_features
>
>if float, must be non-negative.



In [ ]:
def train_svm(config: Configuration, seed: int = 0) -> float:
    """Creates a SVM based on a configuration and evaluates it on the
    iris-dataset using cross-validation."""
    config_dict = dict(config)

    # some additional magic to deal with the complex gamma hyperparameter
    if "gamma" in config.keys():
        config_dict["gamma"] = config_dict["gamma_value"] if config_dict["gamma"] == "value" else "auto"
        config_dict.pop("gamma_value", None)

    classifier = svm.SVC(**config_dict, random_state=seed)  # here we unpack the configuration and add a seed to the SVC class for reproducibility
    scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
    cost = 1 - np.mean(scores)

    return cost

In [ ]:
config = cs.sample_configuration()

print(f'Training \n{config}\n yields an average cross validation score of {train_svm(config, seed=0)}')

## Combining the Search Space with our Training Pipeline

If we compare this with the original pipeline, this is almost no effort and a drop-in replacement, even for rather complex search spaces. All of the design decisions we deem fit for optimization are now included and explicit. Let`s place this in a nice looking and explicit wrapper class:

In [ ]:

import numpy as np
from ConfigSpace import Categorical, Configuration, ConfigurationSpace, Float, Integer
from ConfigSpace.conditions import InCondition
from sklearn import datasets, svm
from sklearn.model_selection import cross_val_score

from smac import HyperparameterOptimizationFacade, Scenario

# We load the iris-dataset (a widely used benchmark)
iris = datasets.load_iris()


class SVM:
    @property
    def configspace(self) -> ConfigurationSpace:
        # Build Configuration Space which defines all parameters and their ranges
        cs = ConfigurationSpace(seed=0)

        # First we create our hyperparameters
        kernel = Categorical("kernel", ["linear", "poly", "rbf", "sigmoid"], default="poly")
        C = Float("C", (0.001, 1000.0), default=1.0, log=True)
        shrinking = Categorical("shrinking", [True, False], default=True)
        degree = Integer("degree", (1, 5), default=3)
        coef = Float("coef0", (0.0, 10.0), default=0.0)
        gamma = Categorical("gamma", ["auto", "value"], default="auto")
        gamma_value = Float("gamma_value", (0.0001, 8.0), default=1.0, log=True)

        # Then we create dependencies
        use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
        use_coef = InCondition(child=coef, parent=kernel, values=["poly", "sigmoid"])
        use_gamma = InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"])
        use_gamma_value = InCondition(child=gamma_value, parent=gamma, values=["value"])

        # Add hyperparameters and conditions to our configspace
        cs.add_hyperparameters([kernel, C, shrinking, degree, coef, gamma, gamma_value])
        cs.add_conditions([use_degree, use_coef, use_gamma, use_gamma_value])

        return cs

    def train(self, config: Configuration, seed: int = 0) -> float:
        """Creates a SVM based on a configuration and evaluates it on the
        iris-dataset using cross-validation."""
        config_dict = dict(config)
        if "gamma" in config.keys():
            config_dict["gamma"] = config_dict["gamma_value"] if config_dict["gamma"] == "value" else "auto"
            config_dict.pop("gamma_value", None)

        classifier = svm.SVC(**config_dict, random_state=seed)
        scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
        cost = 1 - np.mean(scores)

        return cost


In [ ]:
classifier = SVM()
cs = classifier.configspace
config = cs.sample_configuration()
cost = classifier.train(config, seed=0)


print(f'Training \n{config}\n yields an average cross validation score of {cost}')

## Setting up SMAC and optimize

The only thing required to optimize over this search space on our pipeline is setting up our hyperparameter optimizer SMAC. There is a plethora of things we can do to configure or even extend SMAC to fit our hyperparameter optimization needs, but for now let us stick with a powerful out-of-the-box variant, called the `HyperparameterOptimizationFacade`, that comes with strong default choices for optimizing black box functions.

The setup is devided into two components:
1. The `Scenario` object which allows us to specify general information about the run like the number of trials, maximum `wall_clock_time` or the time for an individual trial (`trial_walltime_limit`) we are willing to spend during our hyperparameter optimization. We can even specify hardware constraints that terminate a trial such as `trial_mem_limit`. All of this obviously depends on our budgetary constraints -- do we have a cluster or only a small laptop that we can use to optimize our pipeline.
2. The actual `Facade` object. This object allows us to tinker with the hyperparameter optimization process and leverage everything SMAC has to offer. (more on that later)

In [ ]:
scenario = Scenario(
        classifier.configspace, # here we pass our search space.
        n_trials=50,  # We want to run max 50 trials (combination of config and seed)

)

smac = HyperparameterOptimizationFacade(
      scenario,
      classifier.train, # here we pass our pipeline function that we want to optimize over.
      overwrite=True
)


And now we are ready to optimize our model pipeline:

In [ ]:
incumbent = smac.optimize()
print(incumbent)

After running the HyperparameterOptimizationFacade, we have tried and tested 50 configuration-seed combinations and eventually found the one configuration that performed best on our task. This configuration is called the incumbent. It is the currently best configuration seen so far. We now know that

 'C': 1.2760639488343621,
  'kernel': 'linear',
  'shrinking': False,

is, given our budget constraints, the best model found for our task. Let uns now calculate the cost of this configuration in the following.

In [ ]:
# Let's calculate the cost of the incumbent
# We obviously have seen this conifguration during tuning, but we may want to make
# see its performance on a different seed from the one that we used during training:
incumbent_cost = smac.validate(incumbent)
print(f"Incumbent cost: {incumbent_cost}")

Incumbent cost: 0.013333333333333308


Nice and simple out of the box - right?

But which configurations did it try so far?

In [ ]:
print(f'What configurations have been executed:\n {smac.runhistory._config_ids} \n\n')
print(f'Which config-seed combinations have been executed, did they succeed and what cost did they incur:')

# Notice, that we guard against variability in our pipeline with multiple seeds that are averaged over
for key, value in smac.runhistory._data.items():
  print(key, value)

## A More Advanced Look Into SMAC

Let us have a slightly closer look at the [`HyperparameterOptimizationFacade`](https://github.com/automl/SMAC3/blob/main/smac/facade/hyperparameter_optimization_facade.py)

The Facade holds many useful default components that are used during Bayesian Optimization and out of the box consists of a random forest model as a surrogate function, an Expected Improvement acquistion function, a LocalAndSortedRandomSearch acquisition maximizer and a Random Sobol initial design; i.e. it samples a share of the configurations at random based on the sobol sequence in the search space to kickstart the Bayesian Optimization.
All of these components are configurable and exchanable to suit the user's preferences; enabling the user to tinker with the BO loop on various levels.

Let's say we wanted to change the number of random configurations to sample during the initial design and wanted to change the exploration/exploitation behaviour by means of the Expected Improvement's `xi` parameter:


In [ ]:
initial_design = HyperparameterOptimizationFacade.get_initial_design(scenario, n_configs=5)
acquisition_function = HyperparameterOptimizationFacade.get_acquisition_function(scenario, xi=0.5)

# Now we use SMAC to find the best hyperparameters
smac = HyperparameterOptimizationFacade(
    scenario,
    classifier.train,
    initial_design=initial_design,
    acquisition_function=acquisition_function,
    overwrite=True
)

In [ ]:
incumbent = smac.optimize()

## Other Facades
There are lot's of different needs when optimizing different hyperparameter optimization problems, which is why there are quite a few other facades tailored to different usecases like the `BlackBoxFacade`, `HyperbandFacade`, `MultiFidelityFacade`, `AlgorithmConfigurationFacade` or `RandomFacade` providing sensible defaults for all of these special cases with the same configurability. Find out more about them and what they are useful for in our [documentation](https://automl.github.io/SMAC3/development/3_getting_started.html#facade)

# More Features
While SMAC is already bundled with a lot of functionallity ranging from single to multi-objective, multi-instance and multi-fidelity optimization, there are a lot of features to accomodate your concrete needs which allow you to tinker with the optimization loop. From custom [Callbacks](https://automl.github.io/SMAC3/development/advanced_usage/1_components.html#callback) at various stages during the optimization loop granting you immediate access, there are other features that shipped with the latest version like the Ask & Tell interface or the abbility to continue an optimization run. These features allow for a more interactive experience and more flexible optimization with the human back in the loop. In the following, we will quickly go over some of these features.

## Ask & Tell

The Ask & Tell interface can be used to write a custom SMAC optimization loop for very specific use cases. The underlying idea is that one can very readily ask configurations of SMAC and actively train them before adding the results back to SMAC.



In [ ]:
from smac.runhistory.dataclasses import TrialValue, TrialInfo

smac = HyperparameterOptimizationFacade(
      scenario,
      classifier.train,
      overwrite=True,
  )

# We can ask SMAC which trials should be evaluated next

for _ in range(10):
    info = smac.ask()   # returns a TrialInfo object

    assert info.seed is not None

    cost = classifier.train(info.config, seed=info.seed)
    value = TrialValue(cost=cost, time=0.5)

    smac.tell(info, value)

# Execute the rest of the optimization loop
incumbent = smac.optimize() # Caution: The user's tials are also counted in `n_trials`


## Callbacks & Continuation

In this combined example, we define a callback that is terminating SMAC prematurely after `stop_after` number of steps by inserting it right after the (internally used) tell at the end of every step of the BO loop. After terminating the first smac instance once the first 10 configurations are executed, we let smac find the runhistory in the current directory automatically and continue from there with the same amount of trials - as configured by the scenario object.

In [ ]:
from smac import Callback
from smac.main.smbo import SMBO

class StopCallback(Callback):
    def __init__(self, stop_after: int):
        self._stop_after = stop_after

    def on_tell_end(self, smbo: SMBO, info: TrialInfo, value: TrialValue) -> bool | None:
        """Called after the stats are updated and the trial is added to the runhistory. Optionally, returns false
        to gracefully stop the optimization.
        """
        if smbo.runhistory.finished == self._stop_after:
            return False

        return None

In [ ]:
# Scenario object specifying the optimization "environment"
# here we assume the target function to be deterministic, which is why we only need one seed!
scenario = Scenario(classifier.configspace, deterministic=True, n_trials=50)
stop_after = 10

# Now we use SMAC to find the best hyperparameters
smac = HyperparameterOptimizationFacade(
    scenario,
    classifier.train,  # We pass the target function here
    callbacks=[StopCallback(stop_after=stop_after)],
    overwrite=True,  # Overrides any previous results that are found that are inconsistent with the meta-data
)

incumbent = smac.optimize()
assert smac.runhistory.finished == stop_after

smac1trials = smac.runhistory._data
print(f'These configurations have been executed in the first smac iteration: {smac1trials}')

scenario = Scenario(classifier.configspace, deterministic=True, n_trials=50)

# Now, we want to continue the optimization
# Make sure, we don't overwrite the last run
smac2 = HyperparameterOptimizationFacade(
    scenario,
    classifier.train,
    overwrite=False,
)

# Check whether we have the same incumbent after 'loading' the latest smac run from disk

assert smac.intensifier.get_incumbent() == smac2.intensifier.get_incumbent()
assert smac2.runhistory.finished == stop_after

# And now we finish the optimization
incumbent2 = smac2.optimize()


In [ ]:
smac2trials = smac2.runhistory._data
print(f'These Trials have been executed in the continuation:\n ')
for key in sorted(set(smac2trials) - set(smac1trials), key = lambda trialkey: trialkey.config_id):
  print(key)

Now we are at the end of the guided part of this tutorial. We suggest that you know try to play around with the code explained above and change some parts with the help of the [SMAC documentation](https://automl.github.io/SMAC3/development/3_getting_started.html). Some things you can try are:
* Exchange the dataset by a more complicated one.
* Exchange the SVM to be configured with a Random Forest. This includes exchanging the model itself, but also adapting the configuration space.
* You can also play around with using different SMAC facades (as discussed above) and see how that influences the performance of your model. This also includes adjusting SMAC's hyperparameters themselves.
* You can also try to implement your own HPO tool by leveragint the ask and tell interface explained above.


This brings us to the end of this small SMAC tutorial. We hope you had a lot of fun trying SMAC out and got some insights into how to use it! For more details checkout the [SMAC documentation](https://automl.github.io/SMAC3/development/3_getting_started.html). If you want to get deeper insights into how an optimization run worked and analyze it, you might also want to have a look at our tool [DeepCave](https://github.com/automl/DeepCAVE), which allows such deep analyses.